In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#importing libraries
import os  #for accessing files and folders
import re  #for regular expresssions
import csv #for csv file
import nltk  #for NLP
import requests   #for webscrapping
import pandas as pd   #for file handling
from bs4 import BeautifulSoup  #for webscrapping
nltk.download('punkt')  #for tokenization
#nltk.download('stopwords')  #for stopwords
#nltk.download('averaged_perceptron_tagger')  #for part of speech tagging
#nltk.download('wordnet')  #for lemmatization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
sections = ['politics', 'world', 'us', 'business', 'tech', 'health', 'entertainment', 'style', 'travel', 'sports']

In [15]:
all_urls = []
url = 'https://www.cnn.com'
#sections = ['politics', 'world', 'us', 'business']

for section in sections:
  data = requests.get(f'{url}/{section}').text
  soup = BeautifulSoup(data, features="html.parser")
  for a in soup.find_all('a', href=True):
      if a['href'] and a['href'][0] == '/' and a['href'] != '#':
          a['href'] = url + a['href']
      all_urls.append(a['href'])

In [16]:
def url_in_article(url, current_year='2024'):
    if url:
        if 'cnn.com/{}/'.format(current_year) in url and '/gallery/' not in url:
            return True
    return False

article_urls = [url for url in all_urls if url_in_article(url)]

In [17]:
article_urls

['https://www.cnn.com/2024/07/28/politics/trump-harris-election-100-days/index.html',
 'https://www.cnn.com/2024/07/28/politics/trump-harris-election-100-days/index.html',
 'https://www.cnn.com/2024/07/27/politics/donald-trump-bitcoin-cryptocurrency/index.html',
 'https://www.cnn.com/2024/07/27/politics/kamala-harris-key-issues/index.html',
 'https://www.cnn.com/2024/07/26/politics/harris-team-netanyahu-meeting-reaction/index.html',
 'https://www.cnn.com/2024/07/27/politics/las-vegas-man-charged-threats-bragg-merchan/index.html',
 'https://www.cnn.com/2024/07/26/politics/trump-netanyahu-meeting-mar-a-lago/index.html',
 'https://www.cnn.com/2024/07/26/politics/peter-strzok-lisa-page-fbi-settlement-doj/index.html',
 'https://www.cnn.com/2024/07/26/politics/fbi-trump-interview-assassination-attempt/index.html',
 'https://www.cnn.com/2024/07/27/politics/harris-house-senate-what-matters/index.html',
 'https://www.cnn.com/2024/07/25/politics/biden-oval-office-address-analysis/index.html',
 '

In [18]:
len(article_urls)

680

In [19]:
import csv

# Create a list to store the data
data = []
for i, url in enumerate(article_urls):
  data.append([f'url{i+1:03}', url])

# Write the data to a CSV file
with open('input.csv', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['URL_ID', 'URL'])
  writer.writerows(data)

In [29]:
class WebsiteScraper:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
            "Accept-Encoding": "gzip, deflate",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "DNT": "1",
            "Connection": "close",
            "Upgrade-Insecure-Requests": "1"
        }     #HTTP header to request web server
        self.positive_words = set()   #empty set for positive words
        self.negative_words = set()   #empty set for positive words
        self.stop_words = set()       #empty set for stop words

                        #creating function to scrape websites
    def scrape_websites(self, urls, url_ids, output_file):
        with open('/content/drive/MyDrive/WebScrapping_NLP_CNN_Articles/MasterDictionary/positive-words.txt', 'r', encoding='latin-1') as file:
            self.positive_words = set(file.read().splitlines())

        with open('/content/drive/MyDrive/WebScrapping_NLP_CNN_Articles/MasterDictionary/negative-words.txt', 'r', encoding='latin-1') as file:
            self.negative_words = set(file.read().splitlines())


        stop_words_dir = '/content/drive/MyDrive/WebScrapping_NLP_CNN_Articles/StopWords'
        stop_words_files = os.listdir(stop_words_dir)
        for file_name in stop_words_files:
            file_path = os.path.join(stop_words_dir, file_name)
            with open(file_path, 'r', encoding='latin-1') as file:
                self.stop_words.update(file.read().splitlines())


        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['url_id', 'url', 'positive_score', 'negative_score', 'polarity_score', 'subjectivity_score',
                             'avg_sentence_length', 'percentage_complex_words', 'fog_index',
                             'avg_words_per_sentence', 'complex_word_count', 'syllable_per_word', 'personal_pronouns', 'average_word_length'])

            """
            Polarity: Polarity lies between [-1,1], -1 defines a negative sentiment and 1 defines a positive sentiment
            Subjectivity: Subjectivity quantifies the amount of personal opinion and factual information contained in the text.
                The higher subjectivity means that the text contains personal opinion rather than factual information.
            Fog Index: Fog index is calculated using the formula 0.4 * (Average Sentence Length + Percentage of Complex Words).
            Syllable Count Per Word: Syllable count per word is calculated by counting the number of vowels present in each word of the text.
            """

            for url, url_id in zip(urls, url_ids):
                page = requests.get(url, headers=self.headers)
                soup1 = BeautifulSoup(page.content, "html.parser")
                soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
                article_content = soup2.find("div", class_="article__content")
                if article_content:
                    text1 = article_content.get_text()


                    #title = soup2.find("div", class_="headline__wrapper").get_text()
                    #text1 = soup2.find("div", class_="article__content").get_text()

                    text1 = re.sub(r'\[[0-9]*\]', '', text1)     #cleaning text
                    text1 = re.sub(r'\d+', '', text1)
                    text1 = re.sub(r'\s+', ' ', text1)
                    text1 = text1.lower()

                    cleaned_text = self.clean_text(text1)
                    sentiment_scores = self.calculate_sentiment_scores(cleaned_text)
                    readability_metrics = self.calculate_readability_metrics(cleaned_text)
                    personal_pronouns = self.calculate_personal_pronouns(text1)
                    average_word_length = self.calculate_average_word_length(cleaned_text)

                    writer.writerow([url_id, url] + list(sentiment_scores) + list(readability_metrics) + [personal_pronouns, average_word_length])


                        #creating function for cleaning text, tokenization
    def clean_text(self, text):
        tokens = nltk.word_tokenize(text)
        cleaned_tokens = [token for token in tokens if token.isalpha() and token not in self.stop_words]
        return cleaned_tokens


                        #creating function for calculating sentiment scores
    def calculate_sentiment_scores(self, text):
        positive_score = sum(1 for word in text if word in self.positive_words)
        negative_score = sum(1 for word in text if word in self.negative_words)
        polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
        subjectivity_score = (positive_score + negative_score) / (len(text) + 0.000001)
        return positive_score, negative_score, polarity_score, subjectivity_score


                        #creating function for calculating readability metrics
                        #nothing but average sentence length, complex words,
                        #etcetre...
    def calculate_readability_metrics(self, text):
        sentences = nltk.sent_tokenize(' '.join(text))
        total_words = len(text)
        total_sentences = len(sentences)
        average_sentence_length = total_words / total_sentences

        complex_words = [word for word in text if len(word) > 2]
        percentage_complex_words = len(complex_words) / total_words
        fog_index = 0.4 * (average_sentence_length + percentage_complex_words)

        average_words_per_sentence = total_words / total_sentences
        complex_word_count = len(complex_words)
        syllable_per_word = sum(len(word) for word in text) / total_words

        return average_sentence_length, percentage_complex_words, fog_index, average_words_per_sentence, complex_word_count, syllable_per_word


                        #creating function for calculating personal pronouns
    def calculate_personal_pronouns(self, text):
        personal_pronouns = re.findall(r'\b(I|we|my|ours|us)\b', text, flags=re.IGNORECASE)
        return len(personal_pronouns)


                        #creating function for calculating average word length
    def calculate_average_word_length(self, text):
        total_characters = sum(len(word) for word in text)
        total_words = len(text)
        average_word_length = total_characters / total_words
        return average_word_length


                        #main function, reading input file
text = pd.read_csv('/content/drive/MyDrive/WebScrapping_NLP_CNN_Articles/input.csv')
urls = text['URL']     #taking all URLs in urls
url_ids = text['URL_ID']      #taking all URL_IDs in url_ids

#caling our classs
scraper = WebsiteScraper()
scraper.scrape_websites(urls, url_ids, 'output.csv')
